In [ ]:
import tensorflow as tf
import helper
from PIL import Image

In [ ]:
%matplotlib inline
import os
from glob import glob
from matplotlib import pyplot
import random
def showImageSample():
    show_n_images = 25
    data_dir = './labeled/'
    glob_obj = glob(os.path.join(data_dir, '0','*.jpeg'))
    digitals_image = helper.get_batch(glob_obj[:show_n_images], 28, 28, 'L')
    pyplot.imshow(helper.images_square_grid(digitals_image, 'L'), cmap='gray')
#showImageSample()

In [ ]:
def model_inputs(image_width, image_height, image_channels, z_dim):
    
    x = tf.placeholder(dtype=tf.float32,shape=(None,image_width,image_height,image_channels),name="inputs_x")
    z = tf.placeholder(dtype=tf.float32,shape=(None,z_dim),name="z_dim")
    learning_rate = tf.placeholder(dtype=tf.float32,shape=(),name="learning_rate")

    return x, z, learning_rate

In [ ]:
def discriminator(images, reuse=False):

    alpha=0.2
    with tf.variable_scope("discriminator",reuse=reuse):
        conv_1 = tf.layers.conv2d(images, 64, 4, strides=2, padding='same')
        conv_1_bn = tf.layers.batch_normalization(conv_1, training=True)
        relu1 = tf.maximum(alpha * conv_1_bn, conv_1_bn)
        # 14x14x64
        
        conv_2 = tf.layers.conv2d(relu1, 128, 4, strides=2, padding='same')
        conv_2_bn = tf.layers.batch_normalization(conv_2, training=True)
        relu2 = tf.maximum(alpha * conv_2_bn, conv_2_bn)
        # 7x7x128
        
        conv_3 = tf.layers.conv2d(relu2, 256, 4, strides=2, padding='same')
        conv_3_bn = tf.layers.batch_normalization(conv_3, training=True)
        relu3 = tf.maximum(alpha * conv_3_bn, conv_3_bn)
        # 4x4x256

        flat = tf.reshape(relu3, (-1, 4*4*256))
        logits = tf.layers.dense(flat, 1)
        out = tf.sigmoid(logits)

    return out, logits


In [ ]:
def generator(z, out_channel_dim=5 , is_train=True):
    alpha=0.2
    with tf.variable_scope('generator',reuse=not(is_train)):
        # Reshape it to start the convolutional stack
        fc = tf.layers.dense(z, 4*4*512)
        
        x = tf.reshape(fc, (-1, 4, 4, 512))
        x = tf.layers.batch_normalization(x, training=is_train)
        relu_x = tf.maximum(alpha * x, x)
        # 4x4x512 now

        #conv2d_1_t = tf.nn.conv2d_transpose(relu_x, kernel, output_shape=[64,7,7,256], strides=[1,2,2,1], padding='SAME')
        conv2d_1_t = tf.layers.conv2d_transpose(relu_x, 256,4, strides=1, padding='valid')
        conv2d_1_t_bn = tf.layers.batch_normalization(conv2d_1_t, training=is_train)
        relu1 = tf.maximum(alpha * conv2d_1_t_bn, conv2d_1_t_bn)
        # 7x7x256 now
        
        conv2d_2_t = tf.layers.conv2d_transpose(relu1, 128, 4, strides=2, padding='same')
        conv2d_2_t_bn = tf.layers.batch_normalization(conv2d_2_t, training=is_train)
        relu2 = tf.maximum(alpha * conv2d_2_t_bn, conv2d_2_t_bn)
        # 14x14x128
        
        # Output layer
        logits = tf.layers.conv2d_transpose(relu2, out_channel_dim, 4, strides=2, padding='same')
        # 28x28x3 now
        out = tf.tanh(logits)

    return out

In [ ]:
def model_loss(input_real, input_z, out_channel_dim):
    # TODO: Implement Function
    g_model = generator(input_z, out_channel_dim)
    d_model_real, d_logits_real = discriminator(input_real)
    d_model_fake, d_logits_fake = discriminator(g_model, reuse=True)

    d_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real)*(1 - 0.1)))
    
    d_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))

    d_loss = d_loss_real + d_loss_fake
    return d_loss, g_loss

In [ ]:
def model_opt(d_loss, g_loss, learning_rate, beta1):
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        d_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)
        
    return d_train_opt, g_train_opt

In [ ]:
import numpy as np
import random

def show_generator_output(sess, n_images, input_z, out_channel_dim, image_mode):
    cmap = None if image_mode == 'RGB' else 'gray'
    z_dim = input_z.get_shape().as_list()[-1]
    example_z = np.random.normal(0, 0.02, size=[n_images, z_dim])

    samples = sess.run(
        generator(input_z, out_channel_dim, False),
        feed_dict={input_z: example_z})
    print(samples.shape)
    images_grid = helper.images_square_grid(samples, image_mode)
    pyplot.imshow(images_grid, cmap=cmap)
    pyplot.show()
    
    
def save_output(sess, n_images, input_z, out_channel_dim, image_mode, path, prefix):
    cmap = None if image_mode == 'RGB' else 'gray'
    z_dim = input_z.get_shape().as_list()[-1]
    example_z = np.random.normal(0, 0.02, size=[n_images, z_dim])

    samples = sess.run(
        generator(input_z, out_channel_dim, False),
        feed_dict={input_z: example_z})
    
    helper.save_images(samples, image_mode,path,prefix)

In [ ]:
import time
def train(epoch_count, batch_size, z_dim, learning_rate, beta1, get_batches, data_shape, data_image_mode,output_path,output_prefix):

    input_real,input_z, lr = model_inputs(*data_shape[1:], z_dim)
    
    out_channel_dim = 1
    if data_image_mode == "RGB" : 
        out_channel_dim = 3
    d_loss, g_loss = model_loss(input_real,input_z,out_channel_dim)
    d_train_opt, g_train_opt = model_opt(d_loss,g_loss,lr,beta1)

    steps = 0
    start = time.clock()
    totoal_start = start
    generate_number = 0
    config = tf.ConfigProto() 
    config.gpu_options.per_process_gpu_memory_fraction = 0.3
    
    with tf.Session(config = config) as sess:
        sess.run(tf.global_variables_initializer())
        patience = 10
        for epoch_i in range(epoch_count):
            for batch_images in get_batches(batch_size):
                # TODO: Train Model
                steps += 1

                batch_images = batch_images * 2
                batch_z = np.random.normal(0, 0.02,  size=(batch_size, z_dim))
                _ = sess.run(d_train_opt, feed_dict={input_real: batch_images, input_z: batch_z, lr :learning_rate})
                _ = sess.run(g_train_opt, feed_dict={input_real: batch_images, input_z: batch_z, lr :learning_rate})
                if steps % 100 == 0 :
                    end = time.clock()
                    train_loss_d = d_loss.eval({input_real: batch_images, input_z: batch_z})
                    train_loss_g = g_loss.eval({input_z: batch_z})
                    print("Time: {:.4f}s ..Epoch {}/{}.setp:{}...".format((end - start),epoch_i+1, epoch_count,steps),
                          "Discriminator Loss: {:.4f}...".format(train_loss_d),
                          "Generator Loss: {:.4f}...".format(train_loss_g),
                          "patience: {:2d}".format(patience))
                    start = time.clock()
                    if steps > 5000 and train_loss_g < train_loss_d and epoch_i > 5:
                        patience = patience - 1
                        if (patience == 0):
                            for i in range(int(20000/100)):
                                save_output(sess,100,input_z,out_channel_dim,data_image_mode,output_path,output_prefix)
                            return
                    else:
                        patience = 10
                if steps % 100 == 0:
                    show_generator_output(sess,4,input_z,out_channel_dim,data_image_mode)
                    
                if int(time.clock() - totoal_start) > 1800:
                    for i in range(20000/100):
                        save_output(sess,100,input_z,out_channel_dim,data_image_mode,output_path,output_prefix)
                    
        show_generator_output(sess,4,input_z,out_channel_dim,data_image_mode)

In [ ]:
if __name__ == '__main__':
    data_dir = './labeled/'
    batch_size = 128
    z_dim = 100
    learning_rate = 0.002
    beta1 = 0.5
    epochs = 20000
    
    output_path = './gen'
    if not os.path.exists(output_path):
        os.mkdir(output_path)

    # for every subdir gen the fake image
    for output_prefix in sorted(os.listdir(data_dir)):
        print('begin : ',output_prefix)
        output_dir_path=os.path.join(output_path,output_prefix)

        if not os.path.exists(output_dir_path):
            os.mkdir(output_dir_path)

        digitals_dataset = helper.Dataset('resized', glob(os.path.join(data_dir, output_prefix+'/*.jpeg')))
        with tf.Graph().as_default():
            train(epochs, batch_size, z_dim, learning_rate, beta1, digitals_dataset.get_batches,
                  digitals_dataset.shape, digitals_dataset.image_mode,output_dir_path,output_prefix)
    print('done')